In [1]:
import pandas as pd
import numpy as np
import json
import re

filename = "42hb_permutation0.json"

with open(filename, 'r') as file:
    obj = json.loads(file.read())
    
strands = obj["vstrands"]
name = obj["name"]

vhToScaf, vhToStap, vhNums = {}, {}, []

for strand in strands:
    num = strand["num"]
    vhNums.append(num)
    vhToScaf[num] = strand["scaf"]
    vhToStap[num] = strand["stap"]

scafEnds = [[vh, i] for vh in vhNums for i, base in enumerate(vhToScaf[vh]) if base[1] == -1 and base[3] != -1]
stapEnds = [[vh, i] for vh in vhNums for i, base in enumerate(vhToStap[vh]) if base[1] == -1 and base[3] != -1]

scafPaths = []
for scafEnd in scafEnds:
    path, curr_vh, curr_base = [], *scafEnd
    next_vh, next_base = vhToScaf[curr_vh][curr_base][2:4]
    while next_base != -1:
        if curr_vh != next_vh:
            path.append([curr_vh, next_vh, curr_base])
        curr_vh, curr_base = next_vh, next_base
        next_vh, next_base = vhToScaf[curr_vh][curr_base][2:4]
    scafPaths.append(path)

stapPaths, i, con = {}, 0, {}
for stapEnd in stapEnds:
    path, curr_vh, curr_base = [], *stapEnd
    next_vh, next_base = vhToStap[curr_vh][curr_base][2:4]
    while next_base != -1:
        if not path:
            path.append([curr_vh, curr_base])
        elif curr_vh != next_vh:
            path[-1].append(curr_base)
            path.append([next_vh, next_base])
        curr_vh, curr_base = next_vh, next_base
        next_vh, next_base = vhToStap[curr_vh][curr_base][2:4]
    path[-1].append(curr_base)
    stapPaths[i] = path
    i += 1

for i in range(len(stapPaths)):
    con_i = []
    for j in range(len(stapPaths[i])):
        segment = stapPaths[i][j]
        if segment[1] > segment[2]:
            stapPaths[i][j] = [segment[0], segment[2], segment[1]]
            con_i.append(0)
        else:
            con_i.append(1)
    con[i] = con_i
scafcross = {}
for path in scafPaths:
    for i in path:
        scafcross.setdefault(i[0], []).append(i[2])
        scafcross.setdefault(i[1], []).append(i[2])

for i in scafcross:
    scafcross[i] = sorted(scafcross[i])

for i in scafcross:
    if len(scafcross) == 2:
        continue
    for k in stapPaths:
        for l, domain in enumerate(stapPaths[k]):
            if i == domain[0] and scafcross[i][1] > domain[1] and scafcross[i][1] < domain[2]:
                stapPaths[k].pop(l)
                if con[k][l] == 0:
                    stapPaths[k].insert(l, [domain[0], scafcross[i][2], domain[2]])
                    stapPaths[k].insert(l + 1, [domain[0], domain[1], scafcross[i][1]])
                    con[k].insert(l, 3)
                else:
                    stapPaths[k].insert(l, [domain[0], domain[1], scafcross[i][1]])
                    stapPaths[k].insert(l + 1, [domain[0], scafcross[i][2], domain[2]])
                    con[k].insert(l, 3)
                break
vertex = {i: [[0, 0] for _ in range(len(stapPaths[i]))] for i in stapPaths}

orderedHelix = {i: sorted([k[1], k[2]] for j in stapPaths for k in stapPaths[j] if k[0] == i) for i in vhNums}
for i in orderedHelix:
    orderedHelix[i] = [item for sublist in orderedHelix[i] for item in sublist]
orderedHelix = {i: sorted([k[1], k[2]] for j in stapPaths for k in stapPaths[j] if k[0] == i) for i in vhNums}

filtered_orderedHelix = {}
for i, helix in orderedHelix.items():
    flattened_helix = [item for sublist in helix for item in sublist]
    if len(flattened_helix) > 1:
        filtered_orderedHelix[i] = flattened_helix

orderedHelix = filtered_orderedHelix
smallest_domains = {}
largest_domains = {}

for helix_n, domains in orderedHelix.items():
    smallest_domains[helix_n] = [helix_n, domains[0], domains[1]]
    largest_domains[helix_n] = [helix_n, domains[-2], domains[-1]]
    
i = 0
dangling_ends = {}  
dangling_ends_b = {}
dangling_ends_e = {}
for j in orderedHelix:
    if j in scafcross and smallest_domains[j][1] > scafcross[j][0]:
        new_vertex_id = i
        dangling_ends[(new_vertex_id, new_vertex_id + 1)] = (smallest_domains[j][1] - scafcross[j][0]+1)*1.08
        dangling_ends_b[j] = new_vertex_id
        i+=1
    for k in range(len(orderedHelix[j]) // 2):
        s = [j, orderedHelix[j][2 * k], orderedHelix[j][2 * k + 1]]
        for m in stapPaths:
            for n, segment in enumerate(stapPaths[m]):
                if segment == s:
                    if con[m][n] == 3:
                        if stapPaths[m][n][1] > stapPaths[m][n + 1][1]:
                            i += 1
                            vertex[m][n] = [i, i + 1]
                        else:
                            vertex[m][n] = [i, i + 1]
                            i += 1
                            con[m][n]=2
                    else:
                        vertex[m][n] = [i, i + 1]
                    i += 1
    if j in scafcross and largest_domains[j][2] < scafcross[j][-1]:
        new_vertex_id = i
        dangling_ends[(new_vertex_id, new_vertex_id + 1)] = (scafcross[j][-1] - largest_domains[j][2]+1)*1.08
        dangling_ends_e[j] = new_vertex_id + 1
        i+=1
    i += 1
keys = list(dangling_ends_b.keys())
for idx in range(0, len(keys) - 1, 2):
    first_key = keys[idx]
    second_key = keys[idx + 1]
    new_key = (dangling_ends_b[first_key], dangling_ends_b[second_key])
    dangling_ends[new_key] = 3.24
    
keys = list(dangling_ends_e.keys())
for idx in range(0, len(keys) - 1, 2):
    first_key = keys[idx]
    second_key = keys[idx + 1]
    new_key = (dangling_ends_e[first_key], dangling_ends_e[second_key])
    dangling_ends[new_key] = 3.24

neighbor = {}
for i in stapPaths:
    list_n = []
    for j, segment in enumerate(stapPaths[i]):
        list_s = []
        index = orderedHelix[segment[0]].index(segment[1])
        for offset in [-2, 2]:
            if 0 <= index + offset < len(orderedHelix[segment[0]]) - 1:
                f = [segment[0], orderedHelix[segment[0]][index + offset], orderedHelix[segment[0]][index + offset + 1]]
                for l, path in stapPaths.items():
                    for m, seg in enumerate(path):
                        if seg == f:
                            list_s.append([l, m])
        list_n.append(list_s)
    neighbor[i] = list_n
df = pd.read_csv("42hb_permutation0.csv")

sequence, state = {}, {}
for _, row in df.iterrows():
    a = row[2]
    s = [1 if i == "A" else 2 if i == "T" else 3 if i == "C" else 4 for i in a]
    #helix_n, base_n = int(row[0][0]), int(row[0][2:-1])
    helix_n, base_n = [int(x) for x in re.findall(r'\d+', row[0])]
    for j, path in stapPaths.items():
        if path[0][0] == helix_n and (path[0][1] == base_n or path[0][2] == base_n):
            sequence[j], state[j] = [], []
            c0, c1 = 0, 0
            for l in path:
                c1 += l[2] - l[1] + 1
                seq = s[c0:c1]
                sequence[j].append(seq[::-1] if l[0] % 2 == 0 else seq)
                state[j].append(0)
                c0 = c1


In [ ]:
import csv

def write_strings_to_file(strings_list, file_name="output.txt"):
    with open(file_name, "w") as file:
        for string in strings_list:
            file.write(f"{string}\n")

def find_max_depth(value, depth=0):
    if isinstance(value, list):
        if not value:
            return depth
        return max(find_max_depth(x, depth + 1) for x in value)
    else:
        return depth

def generate_nested_type_string(depth):
    if depth == 0:
        return "int"
    else:
        return "std::vector<" + generate_nested_type_string(depth - 1) + ">"

def convert_value_to_cpp_string(value):
    if isinstance(value, list):
        return "{" + ", ".join(convert_value_to_cpp_string(x) for x in value) + "}"
    else:
        return str(value)

def convert_dict_to_cpp_map(input_dict, map_name, custom_key=False, value_type="int"):
    if value_type == "int":
        max_depth = max(find_max_depth(value) for value in input_dict.values())
        nested_type = generate_nested_type_string(max_depth)
    else:
        nested_type = value_type

    if custom_key:
        key_type = "std::vector<int>"
    else:
        key_type = "int"

    cpp_map = f"std::map<{key_type}, {nested_type}> {map_name} = {{\n"

    for key, value in input_dict.items():
        if custom_key:
            key_string = "{" + ", ".join(map(str, key)) + "}"
        else:
            key_string = str(key)
        cpp_map += f"  {{{key_string}, {convert_value_to_cpp_string(value)}}},\n"

    cpp_map += "};"

    return cpp_map

# suppose you have dictionaries stapPaths, con, vertex, sequence, state, neighbor, scafcross, dangling_ends
# otherwise, replace these with your actual dictionaries
stapPaths_cpp = convert_dict_to_cpp_map(stapPaths, "stapPaths")
con_cpp = convert_dict_to_cpp_map(con, "con")
vertex_cpp = convert_dict_to_cpp_map(vertex, "vertex")
sequence_cpp = convert_dict_to_cpp_map(sequence, "sequence")
state_cpp = convert_dict_to_cpp_map(state, "state")
neighbor_cpp = convert_dict_to_cpp_map(neighbor, "neighbor")
scafcross_cpp = convert_dict_to_cpp_map(scafcross, "scafcross")
dangling_ends_cpp = convert_dict_to_cpp_map(dangling_ends, "dangling_ends", custom_key=True, value_type="double")

write_strings_to_file(
    [stapPaths_cpp, con_cpp, vertex_cpp, sequence_cpp, state_cpp, neighbor_cpp, scafcross_cpp, dangling_ends_cpp],
    "r4.txt",
)
